In [4]:
from ultralytics import YOLO
import os
import random
import cv2
import numpy as np
import shutil
import supervision as sv
import cv2
import matplotlib.pyplot as plt


In [16]:
keypoints = {1: (44, 30), 2: (457, 30), 3: (44, 563), 4: (457, 563), 5: (44, 138), 6: (174, 138), 7: (174, 230), 8: (174, 346), 9: (174, 453), 10: (44, 453), 11: (44, 230), 12: (86, 230), 13: (86, 368), 14: (44, 368), 15: (130, 295), 16: (457, 230), 17: (457, 295), 18: (457, 368), 19: (201, 295), 20: (387, 295), 21: (530, 295), 22: (870, 30), 23: (870, 563), 24: (870, 138), 25: (740, 138), 26: (740, 230), 27: (740, 346), 28: (740, 453), 29: (870, 453), 30: (870, 230), 31: (827, 230), 32: (827, 368), 33: (870, 368), 34: (785, 295), 35: (712, 295)}

In [11]:
# Choose dataset 1 or 2:
dataset = 1  # 1 or 2

if dataset == 1:
    data_folder = "1_train-val_1min_aalesund_from_start"
    yaml_file = "yaml_files/keypoint_1.yaml"
    labels = "labels_1"

else:
    data_folder = "2_train-val_1min_after_goal"
    yaml_file = "yaml_files/keypoint_2.yaml"
    labels = "labels_2"

In [9]:
image_src_folder = f'/work/mbergst/TDT4265_Project/data_yolov8/ball_datasets/{data_folder}/images/all_frames'
label_src_folder = f'/work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/{labels}'
image_dest_folder = f'/work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/{data_folder}/images/'
label_dest_folder = f'/work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/{data_folder}/labels/'

from scripts.keypoint_dataset import distribute_files

distribute_files(image_src_folder, label_src_folder, image_dest_folder, label_dest_folder)

In [12]:
# Last inn sist trente modell
# Gjør det ikke bedre virker det som

def find_latest_model_with_best(base_path, type='keypoints'):
    detect_path = os.path.join(base_path, 'runs', type)
    training_sessions = [os.path.join(detect_path, d) for d in os.listdir(detect_path) if os.path.isdir(os.path.join(detect_path, d))]
    
    # Liste for å holde stiene til alle 'best.pt'-filer
    best_model_paths = []

    for session in training_sessions:
        best_model_path = os.path.join(session, 'weights', 'best.pt')
        if os.path.exists(best_model_path):
            best_model_paths.append((best_model_path, os.path.getmtime(best_model_path)))

    if not best_model_paths:
        print("Ingen 'best.pt' fil funnet i noen av treningsøktene.")
        return None
    
    # Sorter basert på modifiseringstid og returner den nyeste
    best_model_paths.sort(key=lambda x: x[1], reverse=True)
    return best_model_paths[0][0]


base_path = '/work/mbergst/TDT4265_Project' 


model = YOLO(find_latest_model_with_best(base_path, 'keypoints'))
#model = YOLO('yolov8n.pt')

In [1]:
results = model.train(data=yaml_file, epochs=3, batch=20, imgsz=(1600, 900), project='/work/mbergst/TDT4265_Project/runs/keypoints')

In [5]:
# Evaluér modellens prestasjon på valideringssettet
val_results = model.val()

Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24195MiB)
Model summary (fused): 168 layers, 3012473 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/2_train-val_1min_after_goal/labels/val.cache... 361 images, 0 backgrounds, 91 corrupt: 100%|██████████| 361/361 [00:00<?, ?it/s]

val: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/2_train-val_1min_after_goal/images/val/000314.jpg: ignoring corrupt image/label: Label class 38 exceeds dataset class count 35. Possible class labels are 0-34
val: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/2_train-val_1min_after_goal/images/val/000318.jpg: ignoring corrupt image/label: Label class 38 exceeds dataset class count 35. Possible class labels are 0-34
val: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/2_train-val_1min_after_goal/images/val/000319.jpg: ignoring corrupt image/label: Label class 38 exceeds dataset class count 35. Possible class labels are 0-34
val: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov8/keypoint_detection/2_train-val_1min_after_goal/images/val/000324.jpg: ignoring corrupt image/label: Label class 38 exceeds dataset class count 35. Possible class labels are 0-34
val: WARNING ⚠️ /work/mbergst/TDT4265_Project/data_yolov


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:02<00:00, 10.39it/s]


                   all        270       1643      0.761      0.756      0.784      0.573
                   kp2        270        270      0.962      0.963      0.979      0.642
                   kp6        270        132      0.899      0.875      0.946      0.661
                   kp7        270         73      0.898      0.918       0.94      0.517
                   kp8        270          1          0          0          0          0
                  kp16        270        260      0.977      0.962      0.993      0.828
                  kp18        270        259       0.97      0.996      0.994      0.853
                  kp19        270         71       0.83      0.789      0.836      0.476
                  kp20        270        270      0.987      0.956      0.992      0.831
                  kp21        270        270      0.983      0.967      0.992      0.824
                  kp26        270         36      0.863      0.889      0.947      0.669
                  kp2

In [37]:
# Tracking

all_frames = f'/work/mbergst/TDT4265_Project/data_yolov8/ball_datasets/{data_folder}/images/all_frames'
frame_paths = sorted([os.path.join(all_frames, f) for f in os.listdir(all_frames) if f.endswith('.jpg')])


In [38]:
# Iterere over hver frame og bruk track metoden
for frame_path in frame_paths:
    # model.track kan være en generator, så vi bruker en løkke for å hente resultatene
    for result in model.track(frame_path, persist=True, stream=True, line_width=1):
        annotated_frame = result.plot(font_size=1, line_width=1)
        cv2.imshow('frame', annotated_frame)
        # Vent på et tastetrykk i 1 millisekund og sjekk om det er 'ESC'
        key = cv2.waitKey(1) & 0xFF
        if key == 27:  # 27 er ASCII-verdien for 'ESC'
            break

    if key == 27:  # Kontroller igjen om 'ESC' ble trykket for å bryte ut av den ytre løkken
        break

cv2.destroyAllWindows()  # Lukker alle OpenCV-vinduene


image 1/1 /work/mbergst/TDT4265_Project/data_yolov8/ball_datasets/2_train-val_1min_after_goal/images/all_frames/000001.jpg: 928x1600 1 kp2, 1 kp16, 1 kp18, 1 kp20, 1 kp21, 4.8ms
Speed: 12.9ms preprocess, 4.8ms inference, 1.1ms postprocess per image at shape (1, 3, 928, 1600)

image 1/1 /work/mbergst/TDT4265_Project/data_yolov8/ball_datasets/2_train-val_1min_after_goal/images/all_frames/000002.jpg: 928x1600 1 kp2, 1 kp16, 1 kp18, 1 kp20, 1 kp21, 2.8ms
Speed: 3.9ms preprocess, 2.8ms inference, 1.0ms postprocess per image at shape (1, 3, 928, 1600)

image 1/1 /work/mbergst/TDT4265_Project/data_yolov8/ball_datasets/2_train-val_1min_after_goal/images/all_frames/000003.jpg: 928x1600 1 kp2, 1 kp16, 1 kp18, 1 kp20, 1 kp21, 3.1ms
Speed: 4.5ms preprocess, 3.1ms inference, 0.6ms postprocess per image at shape (1, 3, 928, 1600)

image 1/1 /work/mbergst/TDT4265_Project/data_yolov8/ball_datasets/2_train-val_1min_after_goal/images/all_frames/000004.jpg: 928x1600 1 kp2, 1 kp16, 1 kp18, 1 kp20, 1 kp21